In [67]:
# Install needed packages (if not already installed)
!pip install plotly pandas requests

# Import libraries
import pandas as pd
import requests
import plotly.express as px


In [68]:
url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/" \
                "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

# Load confirmed cases data into a DataFrame
df_confirmed = pd.read_csv(url_confirmed)
print(df_confirmed.head())


  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/23  3/1/23  3/2/23  3/3/23  \
0        0        0        0        0  ...   209322  209340  209358  209362   
1        0        0        0        0  ...   334391  334408  334408  334427   
2        0        0        0        0  ...   271441  271448  271463  271469   
3        0        0        0        0  ...    47866   47875   47875   47875   
4        0        0        0        0  ...   105255  105277  105277  105277   

   3/4/23  3/5/23  3/6/23  3/7/23  3/8/23  3/9/23  
0  209369  209390  209406  2

In [69]:
# Melt dataframe to long format
df_long = df_confirmed.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='Date',
    value_name='Confirmed'
)

# Convert Date to datetime
df_long['Date'] = pd.to_datetime(df_long['Date'])
df_long.head()


/tmp/ipython-input-69-2080180496.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0


In [70]:
country = 'India'
df_country = df_long[df_long['Country/Region'] == country]

fig = px.line(df_country, x='Date', y='Confirmed', title=f'COVID-19 Confirmed Cases Over Time in {country}')
fig.show()



In [71]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
import pandas as pd


In [72]:
countries = df_long['Country/Region'].unique()
dropdown = widgets.Dropdown(options=sorted(countries), description='Country:')
display(dropdown)


Dropdown(description='Country:', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antarctic…

In [73]:
def plot_country(country):
    df_country = df_long[df_long['Country/Region'] == country]
    fig = px.line(df_country, x='Date', y='Confirmed', title=f'COVID-19 Confirmed Cases Over Time in {country}')
    fig.show()


In [74]:
widgets.interact(plot_country, country=dropdown)


interactive(children=(Dropdown(description='Country:', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra'…

<function __main__.plot_country(country)>

In [75]:
# Deaths data
url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/" \
             "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

# Recovered data
url_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/" \
                "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

df_deaths = pd.read_csv(url_deaths)
df_recovered = pd.read_csv(url_recovered)


In [76]:
# Melt deaths data
df_deaths_long = df_deaths.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='Date',
    value_name='Deaths'
)

# Melt recovered data
df_recovered_long = df_recovered.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='Date',
    value_name='Recovered'
)

# Convert date to datetime
df_deaths_long['Date'] = pd.to_datetime(df_deaths_long['Date'])
df_recovered_long['Date'] = pd.to_datetime(df_recovered_long['Date'])


/tmp/ipython-input-76-3365844382.py:16: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-76-3365844382.py:17: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [77]:
def plot_country_all(country):
    df_c = df_long[df_long['Country/Region'] == country].groupby('Date')['Confirmed'].sum().reset_index()
    df_d = df_deaths_long[df_deaths_long['Country/Region'] == country].groupby('Date')['Deaths'].sum().reset_index()
    df_r = df_recovered_long[df_recovered_long['Country/Region'] == country].groupby('Date')['Recovered'].sum().reset_index()

    # Merge all into one dataframe
    df_combined = df_c.merge(df_d, on='Date').merge(df_r, on='Date')

    # Plot
    fig = px.line(df_combined, x='Date', y=['Confirmed', 'Deaths', 'Recovered'],
                  title=f'COVID-19 Trends in {country}',
                  labels={"value": "Number of Cases", "variable": "Case Type"})

    fig.show()


In [78]:
widgets.interact(plot_country_all, country=dropdown)


interactive(children=(Dropdown(description='Country:', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra'…

<function __main__.plot_country_all(country)>